## Ejemplo básico de uso de la Assistants API de Azure OpenAI con el SDK de Python.

Requisitos:
- Paquete `openai` instalado (vía `uv add openai`).

- Variables de entorno configuradas:
    AZURE_OPENAI_ENDPOINT       -> https://<tu-recurso>.openai.azure.com
    AZURE_OPENAI_API_KEY        -> clave del recurso de Azure OpenAI
    AZURE_OPENAI_API_VERSION    -> 2024-05-01-preview  (u otra compatible)
    AZURE_OPENAI_ASSISTANT_DEPLOYMENT -> nombre del deployment del modelo (por ejemplo, gpt-4.1-mini)

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)

client = OpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    base_url="https://aoai-lil-demo.openai.azure.com/openai/v1/",
)

In [ ]:
response = client.responses.create(   
  model="gpt-4.1-mini", # Replace with your model deployment name 
  input="what is the Azure openAI responses API?",
)

In [ ]:
print(response.model_dump_json(indent=2))

#### Recuperar una respuesta

In [ ]:
response = client.responses.retrieve("resp_0c1d0b0fdb2b3a5b00693f117bd6c48197b5e9020246da11cc")
print(response.model_dump_json(indent=2))

#### Borrar una respuesta

In [ ]:
response = client.responses.delete("resp_0c1d0b0fdb2b3a5b00693f117bd6c48197b5e9020246da11cc")

print(response)

### 1. Primera llamada: pregunta inicial

In [ ]:
response_1 = client.responses.create(
    model="gpt-4.1-mini",  # Reemplaza con el nombre de tu despliegue de modelo
    input=[
        {"role": "system", "content": "Eres un asistente técnico que responde en un párrafo corto."},
        {"role": "user", "content": "Explícame brevemente qué es Azure OpenAI."},
    ],
    store=True,  # le decimos a la API que guarde el estado de esta conversación
)

print("=== Primera respuesta ===")
print(response_1.output_text)
print(f"ID de la respuesta: {response_1.id}")

### 2. Segunda llamada: continuar la conversación

In [ ]:
response_2 = client.responses.create(
    model="gpt-4.1-mini",  # Reemplaza con el nombre de tu despliegue de modelo
    # Reutilizamos el contexto anterior sin volver a mandar todo el histórico
    previous_response_id=response_1.id,
    input=[
        {
            "role": "user",
            "content": "Ponme un ejemplo de cómo podría usar Azure OpenAI en una empresa de retail.",
        }
    ],
)

print("\n=== Segunda respuesta (con contexto) ===")
print(response_2.output_text)


### 3. Utilizar herramientas (Code Interpreter)

In [ ]:
prompt = """
Tienes estos datos de ventas mensuales:

Enero: 120
Febrero: 135
Marzo: 160
Abril: 155

1. Calcula la media de ventas.
2. Dibuja una pequeña tabla o resumen.
3. Explica brevemente si ves una tendencia de crecimiento.
"""

In [ ]:
response = client.responses.create(
    model="gpt-4.1-mini",  # Reemplaza con el nombre de tu despliegue de modelo
    input=prompt,
    instructions=(
        "Eres un analista de datos. Si lo consideras útil, usa la herramienta de código "
        "para hacer los cálculos antes de responder."
    ),
    tools=[
        {
            "type": "code_interpreter",
            "container": {"type": "auto"}
        }
    ],
    max_output_tokens=500,
)

print(response.output_text)

In [ ]:
print(response.model_dump_json(indent=2))

### Evaluaciones con Azure OpenAI

In [ ]:
from azure.ai.evaluation import RelevanceEvaluator, AzureOpenAIModelConfiguration

load_dotenv(override=True)

# model_config normalmente incluye información del modelo Azure OpenAI
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    azure_deployment="gpt-4.1-mini",
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)

In [ ]:
query = "¿Cuál es la capital de Francia?"
response = "La capital de Francia es París."

relevance_eval = RelevanceEvaluator(model_config)
result = relevance_eval(query=query, response=response)


# Print the result in a more detailed way
print("=== Detalles de la evaluación de relevancia ===")
for key, value in result.items():
    print(f"{key}: {value}")


In [ ]:
from azure.ai.evaluation import GroundednessEvaluator

groundedness_eval = GroundednessEvaluator(model_config)

conversation = {
    "messages": [
        { "content": "¿Qué cineasta dirigió 'Moonrise Kingdom'?", "role": "user" },
        { "content": "Wes Anderson dirigió 'Moonrise Kingdom'.", "role": "assistant", "context": "Wes Anderson  ha dirigido varias películas, incluyendo 'Moonrise Kingdom'." },
        { "content": "¿En qué año se estrenó la película de Wes Anderson Moonrise Kingdom?", "role": "user" },
        { "content": "'Moonrise Kingdom' se estrenó en 2012.", "role": "assistant", "context": "'Moonrise Kingdom' fue lanzada en 2012." },
    ]
}

groundedness_conv_score = groundedness_eval(conversation=conversation)

In [ ]:
import json
print("=== Evaluación de Groundedness de la conversación ===")
print(json.dumps(groundedness_conv_score, indent=4))

In [ ]:
from azure.ai.evaluation import evaluate

# Suponemos que ya has creado groundedness_eval y relevance_eval
result = evaluate(
    data="eval_dataset_3.jsonl",
    evaluators={
        "groundedness": groundedness_eval,
        "relevance": relevance_eval,
    },
    evaluator_config={
        "groundedness": {
            "column_mapping": {
                "query": "${data.query}",
                "context": "${data.context}",
                "response": "${data.response}",
            }
        }
    },
    # Opcional: volcar resultados en un JSON local
    output_path="./myevalresults.json",
)
